In [1]:
%pip install ../..

Processing /home/sjoshi/spuco
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for spuco: filename=spuco-0.0.1-py3-none-any.whl size=68528 sha256=5dfaff04a7cd09c4e7ccde6c980606b24bfb8e44251ae1309ffdb5f28aef3110
  Stored in directory: /tmp/pip-ephem-wheel-cache-z8aejec9/wheels/ef/5d/43/a265894b1d52121a51705a208277e8d9a9670e95fa1a2e7ae6
Successfully built spuco
  Attempting uninstall: spuco
    Found existing installation: spuco 0.0.1
    Uninstalling spuco-0.0.1:
      Successfully uninstalled spuco-0.0.1
Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch 

device = torch.device("cuda:7")

In [3]:
from spuco.invariant_train import ERM
from spuco.datasets import SpuCoMNIST, SpuriousFeatureDifficulty
import torchvision.transforms as T

classes = [[0, 1], [2, 3], [4, 5], [6, 7], [8, 9]]
difficulty = SpuriousFeatureDifficulty.MAGNITUDE_MEDIUM

trainset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=difficulty,
    spurious_correlation_strength=0.995,
    classes=classes,
    split="train"
)
trainset.initialize()

testset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=difficulty,
    classes=classes,
    split="test"
)
testset.initialize()


TypeError: SpuCoMNIST.__init__() got an unexpected keyword argument 'train'

In [ ]:
T.ToPILImage()(trainset[1000][0]).resize((28,28))

In [ ]:
from spuco.models import model_factory 

model = model_factory("lenet", trainset[0][0].shape, trainset.num_classes).to(device)

In [ ]:
from torch.optim import SGD

erm = ERM(
    model=model,
    num_epochs=5,
    trainset=trainset,
    batch_size=32,
    optimizer=SGD(model.parameters(), lr=1e-3, weight_decay=5e-4),
    device=device,
    verbose=True
)

In [ ]:
erm.train()

In [ ]:
from spuco.evaluate import Evaluator

evaluator = Evaluator(
    testset=testset,
    group_partition=testset.group_partition,
    group_weights=trainset.group_weights,
    batch_size=64,
    model=model,
    device=device,
    verbose=True
)
evaluator.evaluate()

In [ ]:
evaluator.worst_group_accuracy

In [ ]:
evaluator.average_accuracy

In [ ]:
evaluator.evaluate_spurious_task()

In [ ]:
import random 
from torch.utils.data import Subset

classes = [[0, 1], [2, 3], [4, 5], [6, 7], [8, 9]]

group_balanced_set = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=difficulty,
    spurious_correlation_strength=0.2,
    classes=classes,
    split="train"
)
group_balanced_set.initialize()
group_balanced_set = Subset(group_balanced_set, [i for key in group_balanced_set.group_partition.keys() for i in group_balanced_set.group_partition[key][:240]])

In [ ]:
from spuco.finetune import DFR 
from torch.optim import SGD

dfr = DFR(
    group_balanced_dataset=group_balanced_set,
    model=model,
    num_epochs=10000,
    lr=1,
    weight_decay=1e-3,
    verbose=True,
    device=device
)

In [ ]:
dfr.train()

In [ ]:
from spuco.evaluate import Evaluator

evaluator = Evaluator(
    testset=testset,
    group_partition=testset.group_partition,
    group_weights=trainset.group_weights,
    batch_size=64,
    model=model,
    device=device,
    verbose=True
)
evaluator.evaluate()

In [ ]:
evaluator.worst_group_accuracy